# What is Polymorphism?

We saw polymorphism last time as the ability to tun the same methods on different objects, either through inheritance or by just defining an ad-hoc protocol (duck typing).

The more general definition is:

**The ability to write code that looks similar, but operates on different types.**

In other words, a single interface serves entities of different types. 


Python type system is strong and dynamic:

- strong: everything has a well-defined type: `type`, `isinstance`
- dynamic: type is not explicitly declared, changes with content

In classic dynamically typed languages such as python is one, most common code is polymorphic. The types of values are restricted only by explicit runtime checks or errors due to failed support for operations at run time. 

Polymorphism is often combined with inheritance, but does not need to be.

One classification of Polymorphism (look up wikipedia) divides it 
- on one axis into adhoc, parametric, and subtype based, and
- on another axis into dynamic(run time) and static(compile time).

### Static vs Dynamic

Dynamic (run-time) polymorphism can be thought of as table based dispatch: that there is, somewhere, atleast conceptually, a table of types, or a linkage of such tables created by inheritance, where implementations are looked up for types.

In static polymorphism found in C++ for example, the binding to the appropriate class can be done at compile time.

In [17]:
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])

In [18]:
class FrenchDeck():
    ranks = [str(n) for n in range(2, 11)] + list('JQKA') 
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
                       
    def __len__(self):
        return len(self._cards)
                       
    def __getitem__(self, position): 
        return self._cards[position]
        

In [19]:
afd = FrenchDeck()
print(len(afd))

52


In [23]:
vars(afd)

{'_cards': [Card(rank='2', suit='spades'),
  Card(rank='3', suit='spades'),
  Card(rank='4', suit='spades'),
  Card(rank='5', suit='spades'),
  Card(rank='6', suit='spades'),
  Card(rank='7', suit='spades'),
  Card(rank='8', suit='spades'),
  Card(rank='9', suit='spades'),
  Card(rank='10', suit='spades'),
  Card(rank='J', suit='spades'),
  Card(rank='Q', suit='spades'),
  Card(rank='K', suit='spades'),
  Card(rank='A', suit='spades'),
  Card(rank='2', suit='diamonds'),
  Card(rank='3', suit='diamonds'),
  Card(rank='4', suit='diamonds'),
  Card(rank='5', suit='diamonds'),
  Card(rank='6', suit='diamonds'),
  Card(rank='7', suit='diamonds'),
  Card(rank='8', suit='diamonds'),
  Card(rank='9', suit='diamonds'),
  Card(rank='10', suit='diamonds'),
  Card(rank='J', suit='diamonds'),
  Card(rank='Q', suit='diamonds'),
  Card(rank='K', suit='diamonds'),
  Card(rank='A', suit='diamonds'),
  Card(rank='2', suit='clubs'),
  Card(rank='3', suit='clubs'),
  Card(rank='4', suit='clubs'),
  Card(r

In [24]:
dir(afd)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cards',
 'ranks',
 'suits']

In [22]:
vars(afd.__class__)

mappingproxy({'ranks': ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A'], '__init__': <function FrenchDeck.__init__ at 0x105939048>, '__weakref__': <attribute '__weakref__' of 'FrenchDeck' objects>, '__doc__': None, '__module__': '__main__', '__getitem__': <function FrenchDeck.__getitem__ at 0x105939840>, 'suits': ['spades', 'diamonds', 'clubs', 'hearts'], '__dict__': <attribute '__dict__' of 'FrenchDeck' objects>, '__len__': <function FrenchDeck.__len__ at 0x1059397b8>})

In [25]:
dir(afd.__class__)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'ranks',
 'suits']

## Ad-hoc polymorphism and Object tables

This is the notion that for a given function, a different function is called for args of different types. 

This enables the Python Data model with the dunder methods. If you call `len(arg)` or `iter(arg)`, we delegate to `arg`'s `__len__` or `__iter__` by *looking them up in the table(class) corresponding to arg*. The net effect is that you get different behaviors for different objects. But you are not looking up a table for the operation, but rather a table for the object. You can think of this as *single dispatch*: the `len` is dispatched based on the type of the argument by looking up a table for the argument.

- We group together the notion that an object responds to such messages into a protocol, such as for example, the informal notion that something is a sequence. This is **Duck Typing**. 

Alex Martelli, the coiner of this word, says:

>In Python, this mostly boils down to avoiding the use of isinstance to check the object’s type (not to mention the even worse approach of checking, for example, whether type(foo) is bar—which is rightly anathema as it inhibits even the simplest forms of inheritance!).

### Tables for dispatching on functions

But, you can also dispatch a function based on its argument, with no lookup in that argument's table, but rather in a *table that is associated with the function*. This is also *single dispatch*, but from a different table.

There is no built in support in Python for this, but you can write it on your own by associating a dictionary with multiple types. 

Here is an example modified from Fluent Python 

In [11]:
#this implementation stripped down from the functools library
def dispatcher(func):
    registry={}
    registry[object]=func #the base scenario for object
    def register(cls, func): #register func for type cls
        registry[cls]=func
    def dispatch(cls): #dispatch returns the function for a type cls, else that for object
        f = registry.get(cls, registry[object])
        return f
    def wrapper(*args, **kw):#wrapper calls the recovered function rith arguments
        return dispatch(args[0].__class__)(*args, **kw)
    wrapper.register=register #set an attribute on the wrapper function, so u can use it to register
    return wrapper

In [12]:
import html
def htmlize(obj):
    content = html.escape(repr(obj)) 
    return "<b>{}</b>".format(content)
htmlize = dispatcher(htmlize)

In [13]:
htmlize.__name__

'wrapper'

In [14]:
def htmlize_int(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)
htmlize.register(int, htmlize_int) 
from IPython.display import HTML
HTML(htmlize('hello world'))

In [15]:
HTML(htmlize(23))

The functools module makes this easy for you by defining a @singledispatch decorator which maintains just such a dictionary (and takes care of inheritance based lookups for us...more on that in a bit).

In [6]:
from functools import singledispatch
import collections
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj)) 
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str) 
def _(text):
    content = html.escape(text).replace('\n', '<br>\n') 
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral) 
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple) 
@htmlize.register(collections.abc.Iterable) 
def _(seq):
    inner = '</li>\n<li>'.join([htmlize(item) for item in seq]) 
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

In [7]:
from IPython.display import HTML
a=htmlize([afd, 1,2,3,4,"hello"])
a

'<ul>\n<li><pre>&lt;__main__.FrenchDeck object at 0x105928da0&gt;</pre></li>\n<li><pre>1 (0x1)</pre></li>\n<li><pre>2 (0x2)</pre></li>\n<li><pre>3 (0x3)</pre></li>\n<li><pre>4 (0x4)</pre></li>\n<li><p>hello</p></li>\n</ul>'

In [8]:
HTML(a)

In some languages such as Julia and lisp(multimethods, CLOS), dispatch on the types of multiple arguments (multiple-dispatch) is also allowed; and indeed is key to the language specializing fast algorithms for specific numerical types. This too can be done in Python, but looks fairly un-natural.

![](https://dl.dropboxusercontent.com/u/75194/juliadispatch.png)

### Parametric Polymorphism

**write functions (or types) that are generic "over" other types.**

- This means, for example, a stack that can take either an int or a float or an animal. Notice that this is generally true in a dynamic language such as python where objects are allocated on the heap and its the references or labels or ids that are pushed onto the stack. 

- In C++ this can be done using templates at compile time to optimize the allocation of space, for example.

- The key question to ask here is..does the Stack affect the behavior of an instance of type `T`, a real/integer/animal. If not, you want to use some sort of generic programming (auto but slow in python)

### Subtype Polymorphism

This refers to the polymorphism that we encounter in situations where our language provides subclassing.

- In a language such as C++, this referes to the notion that a dog and a cat can make sounds through an animal pointer. 
- In python one can use duck typing or inheritance. So subtype polymorphism is then just ad-hoc polymorphism plus an augmented lookup in the inheritance hierarchy.


What's this table we talk about? We hinted at it earlier when we did:

In [9]:
afd.__class__.__dict__

mappingproxy({'__dict__': <attribute '__dict__' of 'FrenchDeck' objects>, 'suits': ['spades', 'diamonds', 'clubs', 'hearts'], 'ranks': ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A'], '__init__': <function FrenchDeck.__init__ at 0x1059317b8>, '__doc__': None, '__len__': <function FrenchDeck.__len__ at 0x1059318c8>, '__weakref__': <attribute '__weakref__' of 'FrenchDeck' objects>, '__getitem__': <function FrenchDeck.__getitem__ at 0x105931950>, '__module__': '__main__'})

But What if we dont find a method here? Either this is a runtime error, or we search in the "parent" classes of this class. We can see all such attributes by using `dir`:

In [10]:
dir(afd)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cards',
 'ranks',
 'suits']

And thus notice that this works because it gets sent up:

In [11]:
hash(afd)

274278618

You can see whats upward of the French Deck

In [12]:
FrenchDeck.mro()

[__main__.FrenchDeck, object]

Its Object, and when something like `__hash__` is not found in `FrenchDeck` its sent up to `object`.

Thus, Subtype Polymorphism works by expanding the search from the current classes table to an augmented table formed by going up the inheritance hierarchy.

### ABC's

- Python actually has a more structured form of typing, called **Goosetyping**, which is a generalization of duck-typing with a bit more structure. 
- Goosetyping is used to define **protocols**, such as the notion of a "sequence". 

Clearly, this can be done informally by agreement as well, but there is an advantage, which you might have seen in `numbers.Integral` above, to defining things higher in an inheritance hierarchy and more formally.
Indeed we will soon use ABCs as a vehicle to document interfaces. 

#### Inheritance and ABCs
Back to our cards example from Fluent Python. 

In [35]:
class FrenchDeck():
    ranks = [str(n) for n in range(2, 11)] + list('JQKA') 
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
                       
    def __len__(self):
        return len(self._cards)
                       
    def __getitem__(self, position): 
        return self._cards[position]

In [36]:
issubclass(FrenchDeck, collections.abc.Sequence)

False

No we are not a sequence and thus:

In [38]:
deck=FrenchDeck()
deck.index(Card(rank='7', suit='spades'))

AttributeError: 'FrenchDeck' object has no attribute 'index'

Erm, but that does not work. We did define `__getitem__` and `__len__`, though.

We can go see the source code for the Sequence ABC at https://github.com/python/cpython/blob/master/Lib/_collections_abc.py#L797 .

I reproduce it here:

```python
### SEQUENCES ###


class Sequence(Sized, Reversible, Container):

    """All the operations on a read-only sequence.
    Concrete subclasses must override __new__ or __init__,
    __getitem__, and __len__.
    """

    __slots__ = ()

    @abstractmethod
    def __getitem__(self, index):
        raise IndexError

    def __iter__(self):
        i = 0
        try:
            while True:
                v = self[i]
                yield v
                i += 1
        except IndexError:
            return

    def __contains__(self, value):
        for v in self:
            if v is value or v == value:
                return True
        return False

    def __reversed__(self):
        for i in reversed(range(len(self))):
            yield self[i]

    def index(self, value, start=0, stop=None):
        '''S.index(value, [start, [stop]]) -> integer -- return first index of value.
           Raises ValueError if the value is not present.
        '''
        if start is not None and start < 0:
            start = max(len(self) + start, 0)
        if stop is not None and stop < 0:
            stop += len(self)

        i = start
        while stop is None or i < stop:
            try:
                if self[i] == value:
                    return i
            except IndexError:
                break
            i += 1
        raise ValueError

    def count(self, value):
        'S.count(value) -> integer -- return number of occurrences of value'
        return sum(1 for v in self if v == value)

Sequence.register(tuple)
Sequence.register(str)
Sequence.register(range)
Sequence.register(memoryview)
```

There are some salient points:

- Notice that `__getitem__` is declared as abstract. This means that we must provide an implementation. Which we have! As did we the definition of `__len__`.But implementing a protocol does not mean being a subclass.

- The source code seems to indicate that only `__getitem__` and `__len__` must be defined. But notice that we have `class Sequence(Sized, Iterable, Container)`. This is multiple Inheritance, where the other ABCs are being used to **Mixin** protocols that we want `Sequence` to support. So lets go digging:

In [39]:
abms=set()
abms = abms.union(collections.abc.Sequence.__abstractmethods__)
print(collections.abc.Sequence.mro())
print(collections.abc.Sequence.__abstractmethods__)
print('=======')
for parent in collections.abc.Sequence.mro()[1:-1]:
    print(parent.__name__, parent.mro())
    print(parent.__abstractmethods__)
    print('-----------')
    abms=abms.union(parent.__abstractmethods__)
abms

[<class 'collections.abc.Sequence'>, <class 'collections.abc.Sized'>, <class 'collections.abc.Iterable'>, <class 'collections.abc.Container'>, <class 'object'>]
frozenset({'__getitem__', '__len__'})
Sized [<class 'collections.abc.Sized'>, <class 'object'>]
frozenset({'__len__'})
-----------
Iterable [<class 'collections.abc.Iterable'>, <class 'object'>]
frozenset({'__iter__'})
-----------
Container [<class 'collections.abc.Container'>, <class 'object'>]
frozenset({'__contains__'})
-----------


{'__contains__', '__getitem__', '__iter__', '__len__'}

Notice that we get `__contains__` and `__iter__` for free. `__contains__` uses `__iter__` and `__iter__` needs `__getitem__`. But Our French Deck is not a sequence subclass.

Why are we fixating on that? Patience, little grasshopper...

#### Registration

We can explicitly register to make sure all is good:

In [40]:
collections.abc.Sequence.register(FrenchDeck)
deck = FrenchDeck()
issubclass(FrenchDeck, collections.abc.Sequence), isinstance(deck, collections.abc.Sequence)

(True, True)

AHA. The notion of registration gets `issubclss` and `isinstance` to work. But even with registration, nothing is inherited: see the mro:

In [27]:
FrenchDeck.mro()

[__main__.FrenchDeck, object]

In [41]:
deck.index(Card(rank='7', suit='spades'))

AttributeError: 'FrenchDeck' object has no attribute 'index'

Such a class is called a **virtual subclass**, and it is not checked for ABC conformance.

### subclassing from ABCs instead of registration

Remember the `Sequence` ABC above seemed to have code...

**Despite being an ABC, it had some method implementations.**

(These are the analogs of C++'s impure virtual functions.)

This suggests that it might be worth inheriting from the ABC instead

In [42]:
class FrenchDeckTake2(collections.abc.Sequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA') 
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
                       
    def __len__(self):
        return len(self._cards)
                       
#     def __getitem__(self, position): 
#         return self._cards[position]



I purposely comment `__getitem__` out. See what happens:

In [43]:
deck = FrenchDeckTake2()

TypeError: Can't instantiate abstract class FrenchDeckTake2 with abstract methods __getitem__

AHA!

By inheriting from an ABC we get runtime checks: *if my "implementation" does not fully implement my "interface", I am in trouble*. 

In [44]:
class FrenchDeckTake3(collections.abc.Sequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA') 
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
                       
    def __len__(self):
        return len(self._cards)
                       
    def __getitem__(self, position): 
        return self._cards[position]


deck = FrenchDeckTake3()

No problems now, and I seem to have picked up some methods from the sequence ABC to boot. Notice that these must be defined in terms of the existing abstract and concrete methods and we will be good...

Subtype polymorphism now works by expanding the search from the current class to other classes in the MRO!

In [45]:
FrenchDeckTake3.mro()

[__main__.FrenchDeckTake3,
 collections.abc.Sequence,
 collections.abc.Sized,
 collections.abc.Iterable,
 collections.abc.Container,
 object]

In [46]:
deck.index(Card(rank='7', suit='spades'))#obtained from sequence implementations

5

As the coder of a concrete subclass, you can override methods inherited from ABCs with more efficient implementations. For example, __contains__ works by doing a linear search, but if you were using a sorted list or binary search tree in your derived class, you could do a binary search using `bisect`.

**So implement a "virtual subclass" by registering if you dont want to use such inherited methods...the parent ABC's wont be in the `mro()`.**

**To get runtime checks and utilize methods that might have been defined in the ABC, subclass.**

#### An interesting edge case

In [48]:
class Answer:
    def __len__(self): 
        return 42
from collections import abc
isinstance(Answer(), abc.Sized), issubclass(Answer, abc.Sized)

(True, True)

In [49]:
Answer.mro()

[__main__.Answer, object]

NO EXPLICIT INHERITANCE HERE!

From https://hg.python.org/cpython/file/3.5/Lib/_collections_abc.py#l300  we can see:

```python
class Sized(metaclass=ABCMeta):

    __slots__ = ()

    @abstractmethod
    def __len__(self):
        return 0

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Sized:
            if any("__len__" in B.__dict__ for B in C.__mro__):
                return True
        return NotImplemented
```

The `isinstance`, `issubclass` dynamically check to see what the abstract methods of the `Sized` ABC are.